In [ ]:
"""
Cell For Papermill Parameters
"""

ARCH = 'resnet18'
NUM_CLASSES = 2
EPOCHS = 50
START_EPOCH = 0
LR = 0.01
MOMENTUM = 0.9
WEIGHT_DECAY = 1e-4
PRINT_FREQ = 10
TRAIN_BATCH= 22
imagesize = 400
WORKERS=4
LR_EPOCH_DROP = 1000
TRAINDIR="../../dataset/ds1/train"
VALDIR="../../dataset/ds1/test"
wb_project = "DNN-Model"
model_save_file = "model_MRI.pth.tar"
cosine_hoops = 2
nn_lock_ly = 0

In [ ]:
import os
import random
import shutil
import time
import pandas as pd
import numpy as np
import math
import json
import glob

import warnings
warnings.filterwarnings("ignore")

import timm
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
from datetime import timedelta
import datetime
import torch.distributed as dist

import wandb
from torch.cuda.amp import GradScaler
from torch.cuda.amp import autocast

import boto3
from botocore.exceptions import NoCredentialsError
from torch.utils.data.sampler import SubsetRandomSampler

import matplotlib.pyplot as plt
%matplotlib inline

import torch.nn.functional as F
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torch import optim
from torchvision import datasets, transforms, models
from torch.autograd import Variable
from torch.utils.data.sampler import SubsetRandomSampler

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, auc
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

from timm.scheduler.cosine_lr import CosineLRScheduler
from timm.scheduler.step_lr import StepLRScheduler

# Config

In [ ]:
GPU=0

SEED=1
VAL_BATCH=TRAIN_BATCH

phone=""

with open('../config/config.json') as config_f:
  data = json.load(config_f)
aws_access_key_id = data['aws_access_key_id']
aws_secret_access_key = data['aws_secret_access_key']
region_name = data['region_name']
WANDB_API_KEY = data['WANDB_API_KEY']

classes_train = os.listdir(TRAINDIR)
classes_valid = os.listdir(VALDIR)

# Weights & Biases

In [ ]:
os.environ["WANDB_API_KEY"] = WANDB_API_KEY
wandb.login()
run = wandb.init(project=wb_project, entity='prostate-cancer', config={"epochs": EPOCHS, "batch_size": TRAIN_BATCH, "momentum": MOMENTUM, 
                   "WEIGHT_DECAY": WEIGHT_DECAY, "arch": ARCH, "TRAINDIR":TRAINDIR,"VALDIR":VALDIR,
                  "imagesize":imagesize, "NUM_CLASSES":NUM_CLASSES,"classes_train":classes_train,"classes_valid":classes_valid})

wandb_run_name = wandb.run.name
wandb_run_id = wandb.run.id

config = wandb.config
config.learning_rate = LR
config.LR_EPOCH_DROP = LR_EPOCH_DROP
config.cosine_hoops = cosine_hoops
config.nn_lock_ly = nn_lock_ly

# Review Dataset

In [ ]:
for class_folder in classes_train:
    print("==================================")
    class_folder = class_folder +'/'
    full_path = os.path.join(TRAINDIR, class_folder)
    print(full_path)
    fileList = glob.glob(full_path +'*jpg*', recursive=True)
    print(len(fileList))

for class_folder in classes_valid:
    print("==================================")
    class_folder = class_folder +'/'
    full_path = os.path.join(VALDIR, class_folder)
    print(full_path)
    fileList = glob.glob(full_path +'*jpg*', recursive=True)
    print(len(fileList))

In [ ]:
def send_sms(phone,text,aws_access_key_id,aws_secret_access_key,region_name):
    # Create an SNS client
    client = boto3.client(
        "sns",
        aws_access_key_id=aws_access_key_id,
        aws_secret_access_key=aws_secret_access_key,
        region_name=region_name
    )

    # Send sms message.
    client.publish(
        PhoneNumber=phone,
        Message=text
    )

In [ ]:
def upload_to_aws(local_file, bucket, s3_file,aws_access_key_id,aws_secret_access_key):
    s3 = boto3.client('s3', aws_access_key_id=aws_access_key_id,
                      aws_secret_access_key=aws_secret_access_key)
    try:
        s3.upload_file(local_file, bucket, s3_file)
        print("Upload Successful")
        return True
    except FileNotFoundError:
        print("The file was not found")
        return False
    except NoCredentialsError:
        print("Credentials not available")
        return False

def download_from_aws(s3_file, bucket,aws_access_key_id,aws_secret_access_key):
    s3 = boto3.client('s3', aws_access_key_id=aws_access_key_id,
                      aws_secret_access_key=aws_secret_access_key)
    try:
        s3.download_file(bucket, s3_file, s3_file)
        print("Download Successful")
        return True
    except FileNotFoundError:
        print("The file was not found")
        return False
    except NoCredentialsError:
        print("Credentials not available")
        return False

In [ ]:
def train(train_loader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter('Time', ':6.3f')
    data_time = AverageMeter('Data', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(train_loader),
        [batch_time, data_time, losses, top1, top5],
        prefix="Epoch: [{}]".format(epoch))

    # Grad Scaler
    scaler = GradScaler()
    # switch to train mode
    model.train()

    end = time.time()
    for i, (images, target) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        if GPU is not None:
            images = images.cuda(GPU, non_blocking=True)
        if torch.cuda.is_available():
            target = target.cuda(GPU, non_blocking=True)

        # compute output
        with autocast():
          output = model(images)
          loss = criterion(output, target)

        # measure accuracy and record loss
        acc1, acc5 = accuracy(output, target, topk=(1, 2))
        losses.update(loss.item(), images.size(0))
        top1.update(acc1[0], images.size(0))
        top5.update(acc5[0], images.size(0))
        
        # use the scaler
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        wandb.log({"Loss/train": loss, 'acc1/train': top1.avg, 'acc5/train': top5.avg})

        if i % PRINT_FREQ == 0:
            progress.display(i)

In [ ]:
def validate(val_loader, model, criterion):
    batch_time = AverageMeter('Time', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(val_loader),
        [batch_time, losses, top1, top5],
        prefix='Test: ')

    # switch to evaluate mode
    model.eval()

    with torch.no_grad():
        end = time.time()
        for i, (images, target) in enumerate(val_loader):
            if GPU is not None:
                images = images.cuda(GPU, non_blocking=True)
            if torch.cuda.is_available():
                target = target.cuda(GPU, non_blocking=True)

            # compute output
            output = model(images)
            loss = criterion(output, target)

            # measure accuracy and record loss
            acc1, acc5 = accuracy(output, target, topk=(1, 2))
            losses.update(loss.item(), images.size(0))
            top1.update(acc1[0], images.size(0))
            top5.update(acc5[0], images.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % PRINT_FREQ == 0:
                progress.display(i)

        print(' * Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f}'
              .format(top1=top1, top5=top5))

    wandb.log({"Loss/val": losses.avg, 'acc1/val': top1.avg, 'acc5/val': top5.avg})
    return top1.avg

In [ ]:
def save_checkpoint(state, is_best, filename_in):
    torch.save(state, filename_in)
    if is_best:
        shutil.copyfile(filename_in, 'model_best_'+wb_project+'_'+wandb_run_name+'_'+str(imagesize)+'.pth.tar')

In [ ]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)

In [ ]:
class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'

In [ ]:
def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    lr = LR * (0.1 ** (epoch // LR_EPOCH_DROP))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [ ]:
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

# Run Model

In [ ]:
random.seed(SEED)
torch.manual_seed(SEED)
cudnn.deterministic = True

In [ ]:
if not torch.cuda.is_available():
    print('GPU not detected.. did you pass through your GPU?')

In [ ]:
torch.cuda.set_device(GPU)
cudnn.benchmark = True

model = models.__dict__[ARCH](pretrained = True)
inf = model.fc.in_features
model.fc = nn.Linear(inf, NUM_CLASSES)

ml = model.cuda(GPU)

In [ ]:
#This freezes layers 1-3 in the total 10 layers of Resnet50
ct = 0
for child in model.children():
    ct += 1
    print(child)
    print(ct)
    if ct < nn_lock_ly:
        for param in child.parameters():
            param.requires_grad = False

In [ ]:
criterion = nn.CrossEntropyLoss().cuda(GPU)

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr = LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)

cosine_restart = EPOCHS/cosine_hoops
cosine_restart = int(cosine_restart - (cosine_restart/(cosine_hoops*2)))

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=cosine_restart,eta_min = 0.000000001)

In [ ]:
transform_train = transforms.Compose([
    transforms.Resize((imagesize,imagesize)),
    transforms.ToTensor(),
])

In [ ]:
transform_val = transforms.Compose([
    transforms.Resize((imagesize,imagesize)),
    transforms.ToTensor(),    
])

In [ ]:
train_dataset = datasets.ImageFolder(TRAINDIR, transform=transform_train)
val_dataset = datasets.ImageFolder(VALDIR, transform=transform_val)

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=TRAIN_BATCH, shuffle=True,num_workers=WORKERS, pin_memory=True, sampler=None)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=VAL_BATCH, shuffle=False,num_workers=WORKERS, pin_memory=True, sampler=None) 

In [ ]:
print(train_loader.dataset.classes)

In [ ]:
%%time
best_acc1 = 0
lrls = []
for epoch in range(START_EPOCH, EPOCHS):
    
    #adjust_learning_rate(optimizer, epoch)

    # train for one epoch
    train(train_loader, model, criterion, optimizer, epoch)

    # evaluate on validation set
    acc1 = validate(val_loader, model, criterion)

    # remember best acc@1 and save checkpoint
    is_best = acc1 > best_acc1
    best_acc1 = max(acc1, best_acc1)

    PATH = "E" +str(epoch) + "_acc1_" +str(round(acc1.item(),2)) + "_" + wandb_run_name + ".tar" 
        
    #save_checkpoint({'epoch': epoch + 1,'arch': ARCH,'state_dict': model.state_dict(),'best_acc1': best_acc1,'optimizer' : optimizer.state_dict(),}, is_best,PATH)    
    #upload_to_aws(PATH, 'prostate-cancer', PATH,aws_access_key_id,aws_secret_access_key)

    lrls.append(scheduler.get_last_lr()[0])      
    lr_float = float(scheduler.get_last_lr()[0])
    scheduler.step()
    
    wandb.log({'lr':lr_float, 'epoch':epoch})

In [ ]:
ax = pd.Series(lrls).plot(logy=True, figsize = (15, 6), title="Learning Rate VS Epoch")
for i in range(0,EPOCHS,1): 
    ax.axvline(i, linewidth=0.01, color='r', linestyle='--')
ax.set_xlabel("Epoch")
ax.set_ylabel("LR (log scale)")
fig = ax.get_figure()
fig.savefig('LR_graph-log.png')
wandb.log({"Media/LR-Graph-Log": wandb.Image("LR_graph-log.png")})

In [ ]:
fig = plt.figure(figsize=(4, 2),frameon =False, dpi=200)  
plt.title('Learning Rate VS Epoch')
plt.plot(lrls)
plt.ylabel('Learning Rate')
plt.xlabel('Epoch')
fn = "LR_graph.png"
plt.savefig(fn,bbox_inches='tight')
plt.show()
plt.close()
wandb.log({"Media/LR-Graph": wandb.Image("LR_graph.png")})

In [ ]:
PATH = model_save_file
save_checkpoint({'epoch': epoch + 1,'arch': ARCH,'state_dict': model.state_dict(),'best_acc1': best_acc1,'optimizer' : optimizer.state_dict(),}, is_best,PATH)    

# RESULTS

## Load Saved Model

In [ ]:
NUM_CLASSES = 2
saved_model_name = PATH
data_dir=VALDIR

In [ ]:
torch.cuda.set_device(GPU)

model = models.__dict__[ARCH](pretrained = True)
inf = model.fc.in_features
model.fc = nn.Linear(inf, NUM_CLASSES)
model.cuda(GPU)

loc = 'cuda:{}'.format(GPU)
checkpoint = torch.load(saved_model_name, map_location=loc)
model.load_state_dict(checkpoint['state_dict'])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
test_transforms = transform_val

In [ ]:
def predict_image(image):
    
    model.eval()
    with torch.no_grad():
    
        image_tensor = test_transforms(image).float()
        image_tensor = image_tensor.unsqueeze_(0)
        input = Variable(image_tensor)
        input = input.to(device)
        output = model(input)
        index = output.data.cpu().numpy().argmax()
       
    return index

In [ ]:
def get_random_images(num,transforms):
    
    data = datasets.ImageFolder(data_dir, transform=transforms)
    classes = data.classes
    indices = list(range(len(data)))
    np.random.shuffle(indices)
    idx = indices[:num]
    
    sampler = SubsetRandomSampler(idx)
    loader = torch.utils.data.DataLoader(data, sampler=sampler, batch_size=num)
    dataiter = iter(loader)
    images, labels = dataiter.next()
    return images, labels, classes

In [ ]:
images, labels, classes = get_random_images(10,transform_val)
wandb.log({"examples-val": [wandb.Image(image) for image in images]})

images, labels, classes = get_random_images(10,transform_train)
wandb.log({"examples-train": [wandb.Image(image) for image in images]})

In [ ]:
counter = 0
while counter < 4:
    to_pil = transforms.ToPILImage()
    images, labels, classes = get_random_images(5,test_transforms)
    fig=plt.figure(figsize=(30,30))
    
    for ii in range(len(images)):
        image = to_pil(images[ii])        
        index = predict_image(image)
        sub = fig.add_subplot(1, len(images), ii+1)
        res = int(labels[ii])
        label_class = int(labels[ii])
        sub.set_title(str(classes[index]) + ":" + classes[label_class])
        plt.axis('off')
        plt.imshow(image)
    plt.savefig("sample.png")
    plt.show()
    counter = counter + 1

In [ ]:
counter = 0
while counter < 4:
    to_pil = transforms.ToPILImage()
    images, labels, classes = get_random_images(5,transform_train)
    fig=plt.figure(figsize=(30,30))
    
    #wandb.log({"examples": [wandb.Image(image) for image in images]})
    
    for ii in range(len(images)):
        image = to_pil(images[ii])        
        index = predict_image(image)
        sub = fig.add_subplot(1, len(images), ii+1)
        res = int(labels[ii])
        label_class = int(labels[ii])
        sub.set_title(str(classes[index]) + ":" + classes[label_class])
        plt.axis('off')
        plt.imshow(image)
    plt.savefig("sample.png")
    plt.show()
    counter = counter + 1

## Confusion Matrix

In [ ]:
actual = []
predicted = []

to_pil = transforms.ToPILImage()
images, labels, classes = get_random_images(500,transform_val)
for ii in range(len(images)):
    image = to_pil(images[ii])
    index = predict_image(image)
    res = int(labels[ii])
    label_class = int(labels[ii])
    actual.append(classes[label_class])
    predicted.append(str(classes[index]))

In [ ]:
cnf_matrix = confusion_matrix(actual, predicted)

In [ ]:
#SOURCE https://runawayhorse001.github.io/LearningApacheSpark/classification.html
import matplotlib.pyplot as plt
import numpy as np
import itertools

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    fig = plt.figure(figsize=(3, 3),frameon =False, dpi=200)  
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    #plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)

    fmt = '.1f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
# Plot non-normalized confusion matrix
plt.figure()
class_names = classes
plot_confusion_matrix(cnf_matrix, classes=class_names,title='Confusion matrix')
plt.savefig("confusion-matrix.png",bbox_inches='tight')
wandb.log({"Media/Confusion Matrix": wandb.Image("confusion-matrix.png")})
plt.show()

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True,title='Normalized confusion matrix')
plt.savefig("confusion-matrix-normalized.png",bbox_inches='tight')
wandb.log({"Media/Normalized Confusion Matrix": wandb.Image("confusion-matrix-normalized.png")})
plt.show()

print(classification_report(actual, predicted, target_names=class_names))
print(accuracy_score(actual, predicted))

# ROC Graph

In [ ]:
def make_ROC_graph(labels_test,prediction):
    """ Text """

    false_positive_rate, recall, thresholds = roc_curve(labels_test,prediction)
    roc_auc = auc(false_positive_rate, recall)
    fig = plt.figure(figsize=(3, 3),frameon =False, dpi=200)  
    plt.title('Receiver Operating Characteristic')
    plt.plot(false_positive_rate, recall, 'b', label='AUC = %0.2f' %roc_auc)
    plt.legend(loc='lower right')
    plt.plot([0, 1], [0, 1], 'r--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    fn = "roc_graph.png"
    plt.savefig(fn,bbox_inches='tight')
    plt.show()
    plt.close()
    wandb.log({"Media/ROC-Graph": wandb.Image("roc_graph.png")})

In [ ]:
actual_b = []
predicted_b = []

for x in actual:
    if x == 'pos':
        actual_b.append(1)
    else:
        actual_b.append(0)

for x in predicted:
    if x == 'pos':
        predicted_b.append(1)
    else:
        predicted_b.append(0)
                
make_ROC_graph(actual_b,predicted_b)

In [ ]:
# Mark the run as finished
wandb.finish()

In [ ]:
sms_text = wandb_run_name + " Is Complete"
send_sms(phone,sms_text,aws_access_key_id,aws_secret_access_key,region_name)